In [1]:
from core.DataLoader import *
from core.models.MLP import * 
from core.models.GAM import *
from core.models.SVM import *
# from core.models.EBM import *

from core.models.RandomForest import *
from core.Visualizer import *
from core.models.Classifier import *
from core.models.XGB import *

# Model eval sf=100 => sf=10

In [2]:
weighted_clf_model_list ={
    "MLP": MLPClassifier,
    "DT": m_DecisionTreeClassifier,
    "RF": m_RandomForestClassifier,
    "LR": m_LogisticRegression,
    "SVM": SVMClassifier,
    "XGB": XGBOOSTClassifier
}
weighted_clf_model_name_list = ['MLP', 'RF(5)', 'RF(U)', 'DT(5)', 'DT(U)', 'LR', 'SVM', 'XGB']
# weighted_clf_model_name_list = ['DT(5)', 'RF(5)', 'MLP']

In [3]:
engine = 'postgres'

dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=['tpch_100'])

In [9]:
# all_features = dl.all_features
all_features = dl.all_features
classification_target = dl.classification_target

# features = dl.base_features
# features = ['left_cardinality', 'sel_of_pred_on_indexed_attr']
print("Using features: ", features)

Using features:  ['left_cardinality', 'base_cardinality', 'sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'sel_of_join_pred', 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed', 'result_size', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr']


In [10]:
# model_perf = {
    
# }
present_idxs = range(len(one_file_dss))
models = {
    'MLP': [],
    'RF(5)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}

scalers = []

for i in present_idxs:  # range(len(one_file_dss)):
    # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
    print(f"Processing index {i} / {len(present_idxs)}")
    ds = one_file_dss[i]
    ds_name = one_file_names[i]
    
    # =========================
    X = ds[features]
    y = ds['optimal_decision']
    X_costs = ds[dl.regression_targets]

    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
    X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
    X_train, X_test, y_train, y_test = \
        X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

    X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)

    # scaler = preprocessing.StandardScaler().fit(X_test)
    X_test = scaler.transform(X_test)

    X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
    
    scalers.append(scaler)
    # =========================
    
    accs = []
    for idx, model_name in enumerate(weighted_clf_model_name_list):
        model = weighted_clf_model_list[model_name.split('(')[0]]        
        if 'DT' in model_name or 'RF' in model_name:
            max_depth = model_name.split('(')[-1].split(')')[0]
            if max_depth.lower() == 'u':
                max_depth = None
            else:
                max_depth = int(max_depth)
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights, max_depth=max_depth)
        else:
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights)
            
        print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
        
        models[model_name].append(clf)
    

  0%|          | 2/500 [00:00<00:25, 19.25it/s]

Processing index 0 / 15
set features: 12


100%|██████████| 500/500 [00:19<00:00, 26.14it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9875
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.95
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9925
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9575
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.99
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.885
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.855


  1%|          | 3/500 [00:00<00:16, 29.69it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9975
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.995
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9925
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.99
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9975
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.98
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.9475


  1%|          | 4/500 [00:00<00:15, 31.60it/s]

Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 0.985
Processing index 14 / 15
set features: 12


100%|██████████| 500/500 [00:20<00:00, 24.94it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 1.0
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 1.0
Accuray of <class 'core.models.SVM.SVMClassifier'>: 1.0
Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 1.0


In [11]:
scale_factor = 10
dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=[f'tpch_{scale_factor}'])

In [12]:
all_features = dl.all_features
classification_target = dl.classification_target

features = dl.base_features
print("Using features: ", features)

Using features:  ['left_cardinality', 'base_cardinality', 'sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'sel_of_join_pred', 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed', 'result_size', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr']


In [13]:
present_idxs = range(len(one_file_dss))
model_accs = {
    'MLP': [],
    'RF(5)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}
for i in present_idxs:  # range(len(one_file_dss)):
    # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
    print(f"Processing index {i} : {one_file_names[i]}")
    ds = one_file_dss[i]
    ds_name = one_file_names[i]
    
    
    # =========================
    X = ds[features]
    y = ds['optimal_decision']    
#     X['left_cardinality'] = X['left_cardinality'] / scale_factor
#     X['base_cardinality'] = X['base_cardinality'] / scale_factor
#     X['result_size'] = X['result_size'] / scale_factor
    
    scaler = scalers[i]
    X = scaler.transform(X.to_numpy())
    y = y.to_numpy()
    # =========================    
    
    for idx, model_name in enumerate(weighted_clf_model_name_list):
        
        clf = models[model_name][i]
        score = clf.score(X, y)    
        print(f"Accuray of {model_name}: {score}")
    
        model_accs[model_name].append(score)

Processing index 0 : tpch_10 customer/postgres_orders_customer_optimal.csv
Accuray of MLP: 0.993
Accuray of RF(5): 0.963
Accuray of RF(U): 0.998
Accuray of DT(5): 0.975
Accuray of DT(U): 0.998
Accuray of LR: 0.884
Accuray of SVM: 0.849
Accuray of XGB: 0.932
Processing index 1 : tpch_10 orders/postgres_customer_orders_optimal.csv
Accuray of MLP: 0.9645
Accuray of RF(5): 0.8825
Accuray of RF(U): 0.9935
Accuray of DT(5): 0.904
Accuray of DT(U): 0.996
Accuray of LR: 0.884
Accuray of SVM: 0.769
Accuray of XGB: 0.924
Processing index 2 : tpch_10 orders/postgres_lineitem_orders_optimal.csv
Accuray of MLP: 0.999
Accuray of RF(5): 0.951
Accuray of RF(U): 1.0
Accuray of DT(5): 0.972
Accuray of DT(U): 1.0
Accuray of LR: 0.902
Accuray of SVM: 0.866
Accuray of XGB: 0.933
Processing index 3 : tpch_10 part/postgres_lineitem_part_optimal.csv
Accuray of MLP: 0.998
Accuray of RF(5): 0.977
Accuray of RF(U): 0.997
Accuray of DT(5): 0.978
Accuray of DT(U): 1.0
Accuray of LR: 0.891
Accuray of SVM: 0.86
Accu

# Model eval sf=10 => sf=1

In [19]:
weighted_clf_model_list ={
    "MLP": MLPClassifier,
    "DT": m_DecisionTreeClassifier,
    "RF": m_RandomForestClassifier,
    "LR": m_LogisticRegression,
    "SVM": SVMClassifier,
    "XGB": XGBOOSTClassifier
}
weighted_clf_model_name_list = ['MLP', 'RF(5)', 'RF(U)', 'DT(5)', 'DT(U)', 'LR', 'SVM', 'XGB']
# weighted_clf_model_name_list = ['DT(5)', 'RF(5)', 'MLP']

In [20]:
engine = 'postgres'

dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=['tpch_10'])

In [21]:
# all_features = dl.all_features
all_features = dl.all_features
classification_target = dl.classification_target

# features = dl.base_features
# features = ['left_cardinality', 'sel_of_pred_on_indexed_attr']
print("Using features: ", features)

Using features:  ['left_cardinality', 'base_cardinality', 'sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'sel_of_join_pred', 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed', 'result_size', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr']


In [22]:
# model_perf = {
    
# }
present_idxs = range(len(one_file_dss))
models = {
    'MLP': [],
    'RF(5)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}

scalers = []

for i in present_idxs:  # range(len(one_file_dss)):
    # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
    print(f"Processing index {i} / {len(present_idxs)}")
    ds = one_file_dss[i]
    ds_name = one_file_names[i]
    
    # =========================
    X = ds[features]
    y = ds['optimal_decision']
    X_costs = ds[dl.regression_targets]

    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
    X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
    X_train, X_test, y_train, y_test = \
        X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

    X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)

    # scaler = preprocessing.StandardScaler().fit(X_test)
    X_test = scaler.transform(X_test)

    X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
    
    scalers.append(scaler)
    # =========================
    
    accs = []
    for idx, model_name in enumerate(weighted_clf_model_name_list):
        model = weighted_clf_model_list[model_name.split('(')[0]]        
        if 'DT' in model_name or 'RF' in model_name:
            max_depth = model_name.split('(')[-1].split(')')[0]
            if max_depth.lower() == 'u':
                max_depth = None
            else:
                max_depth = int(max_depth)
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights, max_depth=max_depth)
        else:
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights)
            
        print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
        
        models[model_name].append(clf)
    

  1%|          | 3/500 [00:00<00:20, 23.83it/s]

Processing index 0 / 15
set features: 12


100%|██████████| 500/500 [00:20<00:00, 24.82it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9775
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9425
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9975
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9575
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.99
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.885
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.855


  1%|          | 4/500 [00:00<00:15, 31.48it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9975
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.99
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9825
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.99
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9975
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.98
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.9475


  1%|          | 4/500 [00:00<00:16, 30.19it/s]

Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 0.985
Processing index 14 / 15
set features: 12


100%|██████████| 500/500 [00:17<00:00, 29.15it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 1.0
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 1.0
Accuray of <class 'core.models.SVM.SVMClassifier'>: 1.0
Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 1.0


In [23]:
# scale_factor = 1
dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=[f'tpch'])

In [24]:
all_features = dl.all_features
classification_target = dl.classification_target

features = dl.base_features
print("Using features: ", features)

Using features:  ['left_cardinality', 'base_cardinality', 'sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'sel_of_join_pred', 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed', 'result_size', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr']


In [25]:
present_idxs = range(len(one_file_dss))
model_accs = {
    'MLP': [],
    'RF(5)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}
for i in present_idxs:  # range(len(one_file_dss)):
    # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
    print(f"Processing index {i} : {one_file_names[i]}")
    ds = one_file_dss[i]
    ds_name = one_file_names[i]
    
    
    # =========================
    X = ds[features]
    y = ds['optimal_decision']    
    
    scaler = scalers[i]
    X = scaler.transform(X.to_numpy())
    y = y.to_numpy()
    # =========================    
    
    for idx, model_name in enumerate(weighted_clf_model_name_list):
        
        clf = models[model_name][i]
        score = clf.score(X, y)    
        print(f"Accuray of {model_name}: {score}")
    
        model_accs[model_name].append(score)

Processing index 0 : tpch customer/postgres_orders_customer_optimal.csv
Accuray of MLP: 0.008
Accuray of RF(5): 0.507
Accuray of RF(U): 0.541
Accuray of DT(5): 0.427
Accuray of DT(U): 0.448
Accuray of LR: 0.082
Accuray of SVM: 0.151
Accuray of XGB: 0.566
Processing index 1 : tpch orders/postgres_customer_orders_optimal.csv
Accuray of MLP: 0.062
Accuray of RF(5): 0.605
Accuray of RF(U): 0.753
Accuray of DT(5): 0.691
Accuray of DT(U): 0.742
Accuray of LR: 0.006
Accuray of SVM: 0.606
Accuray of XGB: 0.679
Processing index 2 : tpch orders/postgres_lineitem_orders_optimal.csv
Accuray of MLP: 0.773
Accuray of RF(5): 0.738
Accuray of RF(U): 0.675
Accuray of DT(5): 0.706
Accuray of DT(U): 0.599
Accuray of LR: 0.134
Accuray of SVM: 0.112
Accuray of XGB: 0.783
Processing index 3 : tpch part/postgres_lineitem_part_optimal.csv
Accuray of MLP: 0.11
Accuray of RF(5): 0.613
Accuray of RF(U): 0.545
Accuray of DT(5): 0.45
Accuray of DT(U): 0.439
Accuray of LR: 0.11
Accuray of SVM: 0.075
Accuray of XGB:

In [ ]:
# MLP = model_accs['MLP'] # [97,96.5,97.5,96.5,96.5,91.5,96.5,96,100,98.5,100,100]
# RF_5 = model_accs['RF(5)'] #[95,95.5,95,96.5,98,90.5,95.5,97,100,98.5,100,100]
# RF_U = model_accs['RF(U)'] #[97.5,97,98.5,98,99,94,97.5,97.5,100,99.5,100,100]
# DT_5 = model_accs['DT(5)'] #[97,97.5,96,97.5,97,90,93,96.5,100,99.5,100,100]
# DT_U = model_accs['DT(U)'] #[98,98.5,96.5,96.5,97,92,95,95.5,100,99,100,100]
# LR = model_accs['LR'] #[92,83.5,92,94,89.5,91,92,90.5,100,96,100,100]
# SVM = model_accs['SVM'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]
# XGB = model_accs['XGB'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]

# d = [DT_5, DT_U,  RF_5, RF_U, XGB, MLP, LR, SVM ]
# result_txt = ''

# for i in range(len(present_idxs)):
#     print(f"{one_file_names[i]}", end='\n')
#     result_txt += f"{one_file_names[i]} \n"
#     for l in d:
#         if l[i] >= 0.9451:
#             s = '& \\textbf{ %.2f }' % l[i]
#         else:
#             s = '& %.2f' % l[i]
#         result_txt +=  s.replace('0.', '.')
#         print(s.replace('0.', '.'), end='')
#     print("\n")
#     result_txt += '\n'
# with open(f'./latex_result/exp1_{engine}_model_acc.txt', 'w') as f:
#     f.write(result_txt)
# print(f"Data written to {f'./latex_result/exp1_{engine}_tpch_model_scale_up.txt'}")